In [31]:
import random
from Bio import SeqIO
import csv

In [32]:
PROMOTER_PATH = "mixed_promoter.fa"
CSV_PATH="dataset_different_size.csv"
ENHANCER_PATH = "enhancer.txt"
MAX_SEQ_ENHANCER_LENGTH=1000

In [33]:
class enhancer:
    def __init__(self, gene, sequence):
        self.gene = gene
        self.sequence = sequence
    def __str__(self):
        return "Gene: "+ str(self.gene) +" sequence: "+ str(self.sequence)+"\n"

class elem:
    def __init__(self, sequence, classe):
        self.sequence = sequence
        self.classe = classe
        
class promoter:
    def __init__(self, nome, sequence):
        self.nome = nome
        self.sequence = sequence

In [34]:
def create_dataset(data,label):
  dataset=[]
  for i in data:
    tmp = elem(i.sequence, label)
    dataset.append(tmp)
  return dataset

In [35]:
with open(ENHANCER_PATH,"r") as file_enhancer:
    enhancer_list = []
    for line in file_enhancer:
      gene, sequence = line.split('\t')
      tmp = enhancer(gene, sequence)
      enhancer_list.append(tmp)

In [36]:
print(f"number of enhancer: {len(enhancer_list)}")

number of enhancer: 418


In [37]:
for item in enhancer_list:
  item.sequence = item.sequence[:-1]

Code part to cut enhancer longer than MAX_SEQ_ENHANCER_LENGTH with this sliding window approach
we create more enchancer from a single one

In [38]:
i = 0
while 1:
  if(i >= len(enhancer_list)) :
    break
  if (len(enhancer_list[i].sequence)> MAX_SEQ_ENHANCER_LENGTH):
    copy = enhancer_list[i].sequence
    tmp = enhancer(enhancer_list[i].gene, copy[MAX_SEQ_ENHANCER_LENGTH:])
    enhancer_list.append(tmp)
    enhancer_list[i].sequence = copy[0:MAX_SEQ_ENHANCER_LENGTH]

  # if(len(enhancer_list[i].sequence)< MAX_SEQ_ENHANCER_LENGTH):
  #   copy = enhancer_list[i].sequence
  #   while (len(copy)< MAX_SEQ_ENHANCER_LENGTH):
  #     copy += 'N'
  #   enhancer_list[i].sequence = copy[0:MAX_SEQ_ENHANCER_LENGTH]
  i += 1

Promoter Handler

In [39]:
promoter_list = []
# Open fasta file to read sequences
with open(PROMOTER_PATH, "r") as handle:
    for record in SeqIO.parse(handle, "fasta"):
       id = record.id
       sequence = record.seq
       tmp = promoter(id, sequence)
       promoter_list.append(tmp)

In [40]:
print(f"number of promoter: {len(promoter_list)}")

number of promoter: 7177


In [41]:
enchancer_dataset=create_dataset(enhancer_list,0)
promoter_dataset=create_dataset(promoter_list,1)

dim_en=len(enchancer_dataset)
dim_prom=len(promoter_dataset)
print(f"promoter dim: {dim_prom}")
print(f"enhancer_dim: {dim_en}")

promoter dim: 7177
enhancer_dim: 8526


In [42]:
"""Balance Dataset """
if dim_en>dim_prom:
    diff=dim_en-dim_prom
    for i in range(diff):
      item = random.choice(promoter_dataset)
      promoter_dataset.append(item)

elif dim_prom>dim_en:
   diff=dim_prom-dim_en
   for i in range(diff):
      item = random.choice(enchancer_dataset)
      enchancer_dataset.append(item)

In [43]:
dataset=promoter_dataset+enchancer_dataset

print(f"lenght of the dataset {len(dataset)}")
"""shuffle the data to have unordered data """
random.shuffle(dataset)

lenght of the dataset 17052


In [44]:
for d in dataset:
    if "" in d.sequence:
        print(d)

In [45]:
data = [(elem.sequence, elem.classe) for elem in dataset]

"""save dataset into a csv file"""

with open(CSV_PATH, "w", newline="") as csvfile:
  csvfile.write("sequence,label\n")
  # Crea un writer CSV
  writer = csv.writer(csvfile)

  # Scrivi la lista nel file CSV
  writer.writerows(data)

# Chiudi il file CSV
csvfile.close()